In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# DBSCAN pT weighted

Lets see what happens to the results of DBSCAN if I weight the clustering by pT

**Load_data**

In [3]:
trk = pd.read_pickle("/media/lucas/QS/l1_nnt/trk_25k.pkl")
mc = pd.read_pickle("/media/lucas/QS/l1_nnt/mc_25k.pkl")

In [4]:
trk.dropna(inplace=True)

**Run the standard stuff first**

In [5]:
from primaryvertexingtools import create_pv_truth_labels

In [6]:
trk = create_pv_truth_labels(trk)

In [7]:
from primaryvertexingtools import PrimaryVertexing

In [8]:
fh_bins = np.linspace(-15, 15, 256)

In [9]:
mydict = {"track_data": trk, "truth_data": mc, "fh_bins": fh_bins, "test_run": False}

In [10]:
PV = PrimaryVertexing(mydict)

Initialized Primary Vertexing setup


In [11]:
PV.run_fh()

100%|███████████████████████████████████| 25000/25000 [00:05<00:00, 4213.16it/s]


Ran Fast Histo


In [12]:
PV.run_dbscan()

100%|████████████████████████████████████| 25000/25000 [01:14<00:00, 336.85it/s]


Ran DBSCAN with eps: 0.08, minPts: 2


# PT weighted DBSCAN

**Test on first event**

In [13]:
e0 = trk.query("entry==0").copy()

In [14]:
e0

trk_pt   trk_eta   trk_phi  trk_d0    trk_z0  trk_fake  \
entry subentry                                                             
0     0         2.623967  1.301414 -0.289234   999.0 -0.878906         2   
      1         3.435026  0.566814 -0.159267   999.0  3.339844         2   
      2         3.774908  1.815973 -0.240375   999.0  4.921875         1   
      3         2.641448 -1.435747 -0.128747   999.0 -0.468750         2   
      4         2.320653  1.837421  0.032604   999.0  4.980469         1   
...                  ...       ...       ...     ...       ...       ...   
      154       2.026653 -2.021094 -1.062191   999.0 -0.410156         2   
      155       2.049097 -2.055119 -0.964472   999.0 -0.410156         2   
      156       2.222179 -2.426809 -0.693084   999.0  5.566406         2   
      157       2.563973  2.081842 -0.530004   999.0 -3.046875         2   
      158       2.303798  1.335547 -0.274176   999.0  4.804688         2   

                is_pv  z0_reco_fh  distance_from_pv_fh  trk_pv_assoc_fh  \
entry subentry                                                            
0     0             0    4.863511             5.742417              0.0   
      1             0    4.863511             1.523667              0.0   
      2             1    4.863511             0.058364              1.0   
      3             0    4.863511             5.332261              0.0   
      4             1    4.863511             0.116958              1.0   
...               ...         ...                  ...              ...   
      154           0    4.863511             5.273667              0.0   
      155           0    4.863511             5.273667              0.0   
      156           0    4.863511             0.702895              0.0   
      157           0    4.863511             7.910386              0.0   
      158           0    4.863511             0.058824              1.0   

                z0_reco_db  trk_pv_assoc_db  
entry subentry                               
0     0           4.980469                0  
      1           4.980469                0  
      2           4.980469                1  
      3           4.980469                0  
      4           4.980469                1  
...                    ...              ...  
      154         4.980469                0  
      155         4.980469                0  
      156         4.980469                0  
      157         4.980469                0  
      158         4.980469                1  

[159 rows x 12 columns]

In [15]:
from sklearn.cluster import DBSCAN

In [16]:
db_clustering = DBSCAN(eps=0.15, min_samples=2).fit(
    e0["trk_z0"].values.reshape(-1, 1), sample_weight=e0["trk_pt"].values
)

In [17]:
e0["db_pt_label"] = db_clustering.labels_

In [18]:
e0.loc[e0["db_pt_label"] < 0, "pt"] = 0

In [19]:
pv_label = (
    e0.groupby(["db_pt_label"])["trk_pt"].sum().sort_values(ascending=False).index[0]
)

In [20]:
e0["db_pt_pv_label"] = 0

In [21]:
e0.loc[e0["db_pt_label"] == pv_label, "db_pt_pv_label"] = 1

In [22]:
z0_reco = np.median(e0.loc[e0["db_pt_pv_label"] == 1, "trk_z0"])

In [23]:
z0_reco

4.9804688

# PT weighted has been implemented in class

In [24]:
from primaryvertexingtools import PrimaryVertexing

In [25]:
mydict = {
    "track_data": trk,
    "truth_data": mc,
    "fh_bins": fh_bins,
    "test_run": False,
}

In [26]:
PVpt = PrimaryVertexing(mydict)

Initialized Primary Vertexing setup


In [27]:
PVpt.run_fh()

100%|███████████████████████████████████| 25000/25000 [00:05<00:00, 4351.07it/s]


Ran Fast Histo


In [28]:
PVpt.run_dbscan(pt_weighted=True)

100%|████████████████████████████████████| 25000/25000 [01:26<00:00, 290.44it/s]


Ran DBSCAN with eps: 0.08, minPts: 2
